In [1]:
import numpy as np
from sklearn.gaussian_process.kernels import Matern, RBF

from doc.mse_estimator import ErrorComparer
from doc.data_generation import gen_rbf_X, gen_matern_X, gen_cov_mat
from doc.plotting_utils import gen_model_barplots
from spe.relaxed_lasso import RelaxedLasso
from spe.tree import Tree
from spe.estimators import kfoldcv, kmeanscv, new_y_est, cp_adaptive_smoother

In [2]:
np.random.seed(1)

In [3]:
## number of realizations to run
niter = 100

## data generation parameters
gsize=10
n=10**2
p=200
s=5
delta = 0.75
snr = 0.4
tr_frac = .5

noise_kernel = 'matern'
noise_length_scale = 1.
noise_nu = .5

X_kernel = 'matern'
X_length_scale = 5.
X_nu = 2.5

## ErrorComparer parameters
alpha = .05
nboot = 100
k = 5
max_depth = 3
lambd = .31
models = [RelaxedLasso(lambd=lambd), Tree(max_depth=max_depth)]
ests = [
    new_y_est,
    new_y_est,
    cp_adaptive_smoother,
    kfoldcv, 
    kmeanscv
]
est_kwargs = [
    {'alpha': None,
    'full_refit': False},
    {'alpha': alpha,
    'full_refit': False},
    {'alpha': alpha, 
    'use_trace_corr': False, 
    'full_refit': False,
    'nboot': nboot},
    {'k': k},
    {'k': k}
]

## plot parameters
model_names = ["Relaxed Lasso", "Decision Tree"]
est_names = ["GenCp", "KFCV", "SPCV"]

In [4]:
err_cmp = ErrorComparer()

In [5]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, gsize, nx)
ys = np.linspace(0, gsize, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

In [6]:
if noise_kernel == 'rbf':
    Sigma_t = gen_cov_mat(c_x, c_y, RBF(length_scale=noise_length_scale))
elif noise_kernel == 'matern':
    Sigma_t = gen_cov_mat(c_x, c_y, Matern(length_scale=noise_length_scale, nu=noise_nu))
else:
    Sigma_t = np.eye(n)
    
Cov_y_ystar = delta*Sigma_t
Sigma_t = delta*Sigma_t + (1-delta)*np.eye(n)

if noise_kernel == 'rbf' or noise_kernel == 'matern':
    Chol_y = np.linalg.cholesky(Sigma_t)
else:
    Chol_y = np.eye(n)

In [7]:
if X_kernel == 'rbf':
    X = gen_rbf_X(c_x, c_y, p)
elif X_kernel == 'matern':
    X = gen_matern_X(c_x, c_y, p, length_scale=X_length_scale, nu=X_nu)
else:
    X = np.random.randn(n,p)

beta = np.zeros(p)
idx = np.random.choice(p,size=s,replace=False)
beta[idx] = np.random.uniform(-1,1,size=s)

In [8]:
tr_idx = np.ones(n, dtype=bool)

# Simulate $Y, Y^* \overset{iid}{\sim} \mathcal{N}(\mu, \Sigma_Y)$

In [9]:
model_errs = []
for model in models:
    errs = err_cmp.compare(
        model,
        ests,
        est_kwargs,
        niter=niter,
        n=n,
        p=p,
        s=s,
        snr=snr, 
        X=X,
        beta=beta,
        coord=coord,
        Chol_y=Chol_y,
        Chol_ystar=Chol_y,
        Cov_y_ystar=None,
        tr_idx=tr_idx,
        fair=False,
        est_sigma=False,
    )
    model_errs.append(errs)

  0%|          | 0/100 [00:00<?, ?it/s]

/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
  1%|          | 1/100 [00:00<00:26,  3.73it/s]/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` ins

In [10]:
fig = gen_model_barplots(
    model_errs, 
    model_names, 
    est_names, 
    title="Adaptive Linear Smoothers: No Shared Noise", 
    has_elev_err=True,
)
fig.show()

# Simulate $\begin{pmatrix} Y \\ Y^* \end{pmatrix} \sim \mathcal{N}\left(\begin{pmatrix} \mu \\ \mu \end{pmatrix}, \begin{pmatrix}\Sigma_Y & \Sigma_{Y, Y^*} \\ \Sigma_{Y^*, Y} & \Sigma_{Y}  \end{pmatrix}\right)$

In [11]:
model_errs = []
for model in models:
    errs = err_cmp.compare(
        model,
        ests,
        est_kwargs,
        niter=niter,
        n=n,
        p=p,
        s=s,
        snr=snr, 
        X=X,
        beta=beta,
        coord=coord,
        Chol_y=Chol_y,
        Chol_ystar=Chol_y,
        Cov_y_ystar=Cov_y_ystar,
        tr_idx=tr_idx,
        fair=False,
        est_sigma=False,
    )
    model_errs.append(errs)

  0%|          | 0/100 [00:00<?, ?it/s]/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning:

`fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.

/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning:

`fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.

  1%|          | 1/100 [00:00<01:07,  1.46it/s]/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning:

`fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.

/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning:

`fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.

In [12]:
fig = gen_model_barplots(
    model_errs, 
    model_names, 
    est_names, 
    title="Adaptive Linear Smoothers: Shared Structured Noise", 
    has_elev_err=True
)
fig.show()